In [1]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import load_model
from sklearn.model_selection import train_test_split
import pickle

In [2]:
def load_and_preprocess_data(preprocess_images=True):
    base_dir = r"D:\Users\RESA\Coding\Alzheimer-Classification-with-CNN-SVM\Notebook\Preprocessing\image_selected.npz"
    
    # Load data from the NPZ file
    loaded_data = np.load(base_dir, allow_pickle=True)
    loaded_combined_slices = loaded_data[list(loaded_data.keys())[0]]

    # Prepare lists for input (X) and output (Y) data
    X = []
    Y = []

    # Create a label mapping for each class
    label_mapping = {'AD': 0, 'CN': 1, 'EMCI': 2, 'LMCI': 3}

    # Iterate through the loaded data
    for label, plane_slices in loaded_combined_slices.item().items():
        for plane, slices in plane_slices.items():
            for selected_slice in slices:
                position, original_slice = selected_slice

                if preprocess_images:
                    # Expand dimensions and repeat for RGB representation
                    original_slice = np.expand_dims(original_slice, axis=-1)
                    original_slice = np.repeat(original_slice, 3, axis=-1)

                    # Min-Max Scaling to bring pixel values to the range [0, 1]
                    min_val = np.min(original_slice)
                    max_val = np.max(original_slice)
                    original_slice = (original_slice - min_val) / (max_val - min_val)

                X.append(original_slice)
                Y.append(label_mapping[label])

    # Convert lists to NumPy arrays
    X = np.array(X) 
    Y = np.array(Y)

    return X, Y

In [3]:
X, Y = load_and_preprocess_data()

In [4]:
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (7200, 224, 224, 3)
Y shape: (7200,)


In [ ]:
from keras import backend as K

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

In [5]:

class_names = ["AD", "CN", "EMCI", "LMCI"]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
def extract_features_for_model(model_name, image):
    if model_name == "DenseNet121":
        # Ekstraksi fitur untuk DenseNet121
        # ...

    elif model_name == "MobileNetV2":
        # Ekstraksi fitur untuk MobileNetV2
        # ...

    elif model_name == "ResNet50":
        # Ekstraksi fitur untuk ResNet50
        # ...

    elif model_name == "VGG16":
        # Ekstraksi fitur untuk VGG16
        model_path = r"D:\Users\RESA\Coding\Evaluasi\VGG16.h5"
        features = extract_vgg16_features(model_path, image)
        return features

    else:
        raise ValueError(f"Model {model_name} tidak dikenali.")


In [7]:
model_paths = [
    r"D:\Users\RESA\Coding\Evaluasi\densenet121_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\densenet121_svm_model_rbf.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\mobilenetv2_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\mobilenetv2_svm_model_rbf.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\resnet50_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\resnet50_svm_model_rbf.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\VGG16_svm_model_poly.pkl",
    r"D:\Users\RESA\Coding\Evaluasi\VGG16_svm_model_rbf.pkl",
]

In [11]:
def load_and_evaluate_model(model_path, X_test, Y_test):
    with open(model_path, 'rb') as model_file:
        model = pickle.load(model_file)
    
    # Ratakan dimensi citra jika diperlukan
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)

    y_true = Y_test
    y_pred_prob = model.predict(X_test_flattened)
    y_pred = np.argmax(y_pred_prob, axis=1)
    
    return y_true, y_pred, y_pred_prob

In [12]:
all_y_true = []
all_y_pred = []
all_y_pred_prob = []

for model_path in model_paths:
    y_true, y_pred, y_pred_prob = load_and_evaluate_model(model_path, X_test, Y_test)
    all_y_true.append(y_true)
    all_y_pred.append(y_pred)
    all_y_pred_prob.append(y_pred_prob)

ValueError: X has 150528 features, but SVC is expecting 1024 features as input.

In [ ]:
for i in range(len(model_paths)):
    cm = confusion_matrix(all_y_true[i], all_y_pred[i])
    report = classification_report(all_y_true[i], all_y_pred[i])

    print(f"\nConfusion Matrix for Scenario {i + 1}:\n{cm}")
    print(f"\nClassification Report for Scenario {i + 1}:\n{report}")

    # Menampilkan plot bar chart untuk confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix - Scenario {i + 1}")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

    # Menampilkan probabilitas prediksi
    plt.figure(figsize=(12, 4))
    for j in range(len(class_names)):
        plt.subplot(1, len(class_names), j + 1)
        plt.hist(all_y_pred_prob[i][:, j], bins=20, label=class_names[j], alpha=0.7)
        plt.title(f"Probability Distribution - {class_names[j]}")
        plt.xlabel("Probability")
        plt.ylabel("Frequency")
        plt.legend()
    plt.tight_layout()
    plt.show()

NameError: name 'all_y_true' is not defined